In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from zipfile import ZipFile

import datetime
import json
import re
from tqdm import tqdm

In [2]:
pathname = "/home/benjamin/Folders_Python/Weight/imports/"

polar_filename = pathname + "TBD.zip"  # données de Polar

# next : aller chercher les archives zip les plus récentes commençant par data_BEN et File-Export...

In [3]:
# Récupère données POIDS

# Source : Withings https://healthmate.withings.com/

# Account > User > Parameters > Download ==> data_BEN_xxxxx.zip ==> contient weight.csv

withings_filename = pathname + "data_BEN.zip"  # données de HealthMate

with ZipFile(withings_filename, 'r') as weight_zip:
    weight_csv = weight_zip.extract('weight.csv', path=pathname)

# extrait la dataframe poids ----------------

colnames = ['Date', 'Poids (kg)', 'Gras (kg)']

df_weight = pd.read_csv(weight_csv, usecols=colnames)
df_weight.rename(columns = {'Poids (kg)' : 'Masse_Totale' , 'Gras (kg)' : 'Masse_Grasse'}, inplace=True)

# transforme le champ str de Date en datetime object

date_format = '%Y-%m-%d %H:%M:%S'

df_weight['Date'] = df_weight['Date'].apply(lambda x : datetime.datetime.strptime(x, date_format).date())
df_weight = df_weight.groupby('Date').mean()

start_date = datetime.date(2020, 8, 1)  # on prend les data depuis le 1er Août 2020

df_weight = df_weight[df_weight.index >= start_date]
df_weight.dropna(inplace=True)

df_weight['Masse_Maigre'] = df_weight['Masse_Totale'] - df_weight['Masse_Grasse']
df_weight['MG%'] = df_weight['Masse_Grasse'] / df_weight['Masse_Totale']
df_weight['BMR'] = 370 + 21.6 * df_weight['Masse_Maigre']

df_weight.sort_index()

,Masse_Totale,Masse_Grasse,Masse_Maigre,MG%,BMR
Date,,,,,
2020-08-09,107.250000,37.190000,70.060000,0.346760,1883.296
2020-08-11,104.650000,33.150000,71.500000,0.316770,1914.400
2020-08-12,104.350000,33.800000,70.550000,0.323910,1893.880
2020-08-13,104.400000,34.340000,70.060000,0.328927,1883.296
2020-08-14,104.500000,33.850000,70.650000,0.323923,1896.040
...,...,...,...,...,...
2022-12-29,71.566667,11.422222,60.144444,0.159603,1669.120
2022-12-30,71.310000,11.210000,60.100000,0.157201,1668.160
2022-12-31,71.810000,10.890000,60.920000,0.151650,1685.872


In [4]:
df_weight.tail(10)

,Masse_Totale,Masse_Grasse,Masse_Maigre,MG%,BMR
Date,,,,,
2022-12-24,72.500000,10.766000,61.734000,0.148497,1703.454400
2022-12-25,72.478571,11.037143,61.441429,0.152281,1697.134857
2022-12-26,71.950000,10.984286,60.965714,0.152666,1686.859429
2022-12-27,72.680000,11.192000,61.488000,0.153990,1698.140800
2022-12-28,72.018750,11.908750,60.110000,0.165356,1668.376000
2022-12-29,71.566667,11.422222,60.144444,0.159603,1669.120000
2022-12-30,71.310000,11.210000,60.100000,0.157201,1668.160000
2022-12-31,71.810000,10.890000,60.920000,0.151650,1685.872000
2023-01-01,71.800000,9.500000,62.300000,0.132312,1715.680000


In [5]:
# Récupère données FOOD

# https://www.myfitnesspal.com/reports

# www.myfitnesspal.com ==> reports > export data ==> File-Export-date1-to-date2.zip

mfp_filename = pathname + "File-Export.zip" # données de My Fitness Pal

# Extrait les archives
    
with ZipFile(mfp_filename, 'r') as food_zip:
    output_dir = food_zip.namelist()
    target = 'Nutrition-Summary'
    for l in output_dir:
        if l[:len(target)] == target:
            food_csv = food_zip.extract(l, path=pathname)
            break
        
# extrait la dataframe food -------------------------------

colnames = ['Date', 'Meal', 'Calories', 'Fat (g)', 'Carbohydrates (g)', 'Protein (g)']

df_food = pd.read_csv(food_csv, usecols=colnames)
df_food.rename(columns = {'Fat (g)' : 'Lipides' , 'Carbohydrates (g)' : 'Glucides', 'Protein (g)' : 'Proteines'}, inplace=True)

# transforme le champ str de Date en datetime object

date_format = '%Y-%m-%d'

df_food['Date'] = df_food['Date'].apply(lambda x : datetime.datetime.strptime(x, date_format).date())
df_food = df_food.groupby('Date').sum()

start_date = datetime.date(2020, 8, 1)  # on prend les data depuis le 1er Août 2020

df_food = df_food[df_food.index >= start_date]
df_food.dropna(inplace=True)
df_food.sort_index()

,Calories,Lipides,Glucides,Proteines
Date,,,,
2020-08-03,1917.0,89.7,194.8,73.0
2020-08-04,1992.7,67.7,267.5,78.4
2020-08-05,1948.9,84.1,213.1,86.9
2020-08-06,1937.2,66.4,132.9,60.0
2020-08-07,1674.0,67.5,166.2,95.2
...,...,...,...,...
2022-12-30,1666.7,36.4,141.3,191.9
2022-12-31,3197.2,66.4,424.4,203.1
2023-01-01,1726.4,38.4,152.0,195.1


In [6]:
df_food.tail(10)

,Calories,Lipides,Glucides,Proteines
Date,,,,
2022-12-25,1850.3,34.7,166.0,160.7
2022-12-26,2328.3,58.3,273.1,173.1
2022-12-27,1611.4,37.5,120.1,189.5
2022-12-28,1767.4,39.0,180.2,166.6
2022-12-29,1784.0,33.7,179.0,189.0
2022-12-30,1666.7,36.4,141.3,191.9
2022-12-31,3197.2,66.4,424.4,203.1
2023-01-01,1726.4,38.4,152.0,195.1
2023-01-02,2044.6,47.1,233.7,173.5


In [7]:
def extract_data_training(exo_dict):
    # utility fonction pour récupérer : date, durée, type exercice et calories dépensées
    
    # récupère date
    pattern = '2(\d){3,3}-(\d){2,2}-(\d){2,2}'  # on cherche une date du type 2xxx-yy-zz
    p = re.compile(pattern)
    m = p.search(exo_dict.get('startTime'))
    if m: 
        time_format = '%Y-%m-%d'
        exo_date = datetime.datetime.strptime( exo_dict.get('startTime')[:10], time_format ).date()
    
    # récupère durée
    pattern = '(\d)+[.]?(\d)*'  # on cherche une durée du type xxxx.yyyy
    p = re.compile(pattern)
    m = p.search(exo_dict.get('duration'))
    if m:
        exo_duration = float(m.group())
    
    # récupère type activité et calories dépensées
    d = exo_dict.get('exercises')[0]   
    exo_type = d.get('sport')
    if d.get('kiloCalories'):
        exo_cals = float(d.get('kiloCalories'))
    else:
        exo_cals = 0.0
    
    return exo_date, exo_duration, exo_type, exo_cals

In [8]:
# Récupère les données EXERCICE

# https://account.polar.com/#export ==> polar-user-data-export-xxxxx.zip

polar_filename = pathname + "polar-user-data-export.zip"  # données de HealthMate

with ZipFile(polar_filename, 'r') as polar_zip:
    output_dir = polar_zip.namelist()
    
    target = 'training-session'
    dict_all_exos = {}
    for i, enr_name in enumerate(tqdm(output_dir)): # on parcourt la liste des archives
        if enr_name[:len(target)] == target:  # si c'est un enregistrement d'une session de training, on traite
            enr_json = polar_zip.extract(enr_name, path=pathname)   # extraction du json
            with open(enr_json, 'r') as f:
                exo_dict = json.load(f)
                exo_date, exo_duration, exo_type, exo_cals = extract_data_training(exo_dict)
                if dict_all_exos.get(exo_date) == None:
                    dict_all_exos[exo_date] = [ exo_duration, exo_cals]
                else:
                    dict_all_exos[exo_date] = [ dict_all_exos[exo_date][0] + exo_duration, dict_all_exos[exo_date][1] + exo_cals ]
                # print(f'exercice {i+1} : date = {exo_date}, durée = {exo_duration}, type = {exo_type}, cals = {exo_cals}')
            os.remove(enr_json)
    
    df_exos = pd.DataFrame.from_dict(dict_all_exos, orient='index', columns=['exo_duree', 'exo_cals_bruts']).sort_index()

100%|██████████| 632/632 [00:12<00:00, 48.76it/s]


In [9]:
df_exos.tail(20)

,exo_duree,exo_cals_bruts
2022-12-08,2160.295,355.0
2022-12-09,4525.056,798.0
2022-12-11,3921.482,538.0
2022-12-12,3660.598,310.0
2022-12-13,3120.471,364.0
2022-12-14,2880.614,310.0
2022-12-15,3970.997,448.0
2022-12-16,3360.712,355.0
2022-12-18,3900.587,548.0
2022-12-20,4380.588,614.0


In [10]:
df_all = pd.concat( [df_weight, df_food, df_exos], axis=1 ).sort_index()

# remplace les NaN dans les colonnes exercice par 0.0 : jour sans exercice
df_all['exo_duree'].fillna(0.0, inplace=True)
df_all['exo_cals_bruts'].fillna(0.0, inplace=True)

# drop les jours où il manque des données autres que les exercices
df_all.dropna(inplace=True)

In [11]:
df_all.head(20)

,Masse_Totale,Masse_Grasse,Masse_Maigre,MG%,BMR,Calories,Lipides,Glucides,Proteines,exo_duree,exo_cals_bruts
2020-08-09,107.250,37.190000,70.060000,0.346760,1883.296,1861.3,71.3,232.5,66.1,0.0,0.0
2020-08-11,104.650,33.150000,71.500000,0.316770,1914.400,1772.4,50.5,185.0,104.1,0.0,0.0
2020-08-12,104.350,33.800000,70.550000,0.323910,1893.880,1894.3,59.3,220.3,87.5,0.0,0.0
2020-08-13,104.400,34.340000,70.060000,0.328927,1883.296,1822.6,71.5,222.6,70.2,0.0,0.0
2020-08-14,104.500,33.850000,70.650000,0.323923,1896.040,1891.5,61.8,216.7,82.5,0.0,0.0
2020-08-15,104.250,34.280000,69.970000,0.328825,1881.352,1925.3,83.5,186.7,68.9,0.0,0.0
2020-08-16,104.050,34.430000,69.620000,0.330899,1873.792,2021.7,92.8,216.2,81.4,0.0,0.0
2020-08-17,103.900,33.880000,70.020000,0.326083,1882.432,1946.5,66.2,232.1,72.5,0.0,0.0
2020-08-18,103.550,34.130000,69.420000,0.329599,1869.472,1789.5,51.9,220.6,100.9,0.0,0.0
2020-08-19,103.250,32.650000,70.600000,0.316223,1894.960,1914.8,69.9,179.6,86.0,0.0,0.0


In [12]:
df_all.describe(include='all')

,Masse_Totale,Masse_Grasse,Masse_Maigre,MG%,BMR,Calories,Lipides,Glucides,Proteines,exo_duree,exo_cals_bruts
count,860.000000,860.000000,860.000000,860.000000,860.000000,860.000000,860.000000,860.000000,860.000000,860.000000,860.000000
mean,74.538276,12.560369,61.977908,0.162046,1708.722810,1991.693372,55.148140,201.428721,153.222674,2572.396637,285.444186
std,9.254239,6.181486,3.154125,0.052715,68.129094,421.760545,20.501535,70.445810,39.625294,2983.443961,318.594513
min,66.000000,7.541667,58.164286,0.112927,1626.348571,249.700000,2.900000,30.200000,16.400000,0.000000,0.000000
25%,68.748214,8.656234,59.879167,0.125747,1663.390000,1802.300000,41.775000,160.375000,126.925000,0.000000,0.000000
50%,70.083333,9.469286,60.724667,0.135347,1681.652800,1871.900000,52.200000,188.350000,155.800000,1926.284500,213.000000
75%,76.867500,13.973333,62.751042,0.183137,1725.422500,2056.900000,65.950000,226.625000,181.500000,4969.528750,540.250000
max,107.250000,37.190000,72.360000,0.346760,1932.976000,5762.000000,235.900000,862.900000,267.100000,28800.000000,2290.000000


In [13]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 860 entries, 2020-08-09 to 2023-01-02
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Masse_Totale    860 non-null    float64
 1   Masse_Grasse    860 non-null    float64
 2   Masse_Maigre    860 non-null    float64
 3   MG%             860 non-null    float64
 4   BMR             860 non-null    float64
 5   Calories        860 non-null    float64
 6   Lipides         860 non-null    float64
 7   Glucides        860 non-null    float64
 8   Proteines       860 non-null    float64
 9   exo_duree       860 non-null    float64
 10  exo_cals_bruts  860 non-null    float64
dtypes: float64(11)
memory usage: 80.6+ KB
